# Imports

In [1]:
# %load_ext autoreload
# %autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
import albumentations as albu
import albumentations.pytorch as albu_pt
from sklearn.metrics import jaccard_score
%matplotlib inline

import apex
import torch
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import pytorch_tools as pt

from src.dataset import OpenCitiesDataset, OpenCitiesTestDataset, InriaTilesDataset, get_dataloaders
from src.augmentations import get_aug
from src.callbacks import ThrJaccardScore
from pytorch_tools.fit_wrapper.callbacks import SegmCutmix
from pytorch_tools.tta_wrapper import TTA

In [3]:
import yaml
from src.utils import MODEL_FROM_NAME
from src.utils import TargetWrapper
from pytorch_tools.fit_wrapper.callbacks import Callback
from pytorch_tools.utils.misc import to_numpy
from src.utils import criterion_from_list
from src.utils import ToCudaLoader

# Get dataloaders

In [6]:
BS = 32
train_dtld_gpu , val_dtld_gpu = get_dataloaders(
    [
        "tier1", 
#         "tier2"
    ],
    batch_size=BS
)


Using datasets: ['tier1']. Train size: 25281. Val size 4381.


# Results exploration

In [19]:
PREV_WEIGHTS = None
PREDS = None
PREDS_TTA = None

MEAN=(0.485, 0.456, 0.406)
STD=(0.229, 0.224, 0.225)

LOADER = iter(val_dtld_gpu)
# LOADER = val_dtld_inria_gpu
IMGS, MASKS = next(LOADER)

SZ = 512
test_loader = DataLoader(OpenCitiesTestDataset(get_aug("test", size=SZ)), batch_size=BS, shuffle=True)
TEST_LOADER = iter(test_loader)
_, IMGS , _ = next(TEST_LOADER)
MASKS = torch.zeros_like(IMGS)
IMGS, MASKS = IMGS.cuda(), MASKS.cuda()

In [20]:
@interact(
    weights=sorted(os.listdir("logs/")),
    N=widgets.IntSlider(min=0, max=BS-1, continuous_update=True),
    thr=widgets.FloatSlider(0.5, min=0.2, max=0.8, step=0.1, continuous_update=False),
)
def foo(weights=None, N=0, thr=0.5, adaptive_thr=False, use_tta=False):
    global PREV_WEIGHTS
    global PREDS
    global PREDS_TTA
#     global PREV_IMGS_MASKS_PREDS
    
    if weights is None:
        print("select weights")
        return 
    
    if weights != PREV_WEIGHTS:
        PREV_WEIGHTS = weights
        log_path = "logs/" + weights + "/"
        config = yaml.load(open(log_path + "config.yaml"))
        model = MODEL_FROM_NAME[config["segm_arch"]](config["arch"], **config.get("model_params", {})).cuda()
        model.load_state_dict(torch.load(log_path + 'model.chpn')["state_dict"], strict=False)
        model.eval()
        tta_model = pt.tta_wrapper.TTA(model, segm=True, h_flip=True, rotation=[90], merge="gmean", activation="sigmoid")
        with torch.no_grad():
            PREDS = model(IMGS).cpu().detach().sigmoid()
            PREDS_TTA = tta_model(IMGS).cpu().detach()#.sigmoid()
        del model
        del tta_model

    img = to_numpy(IMGS[N]).swapaxes(0, 2)
    img = np.clip((img * STD + MEAN), 0, 1)
    mask = to_numpy(MASKS[N]).swapaxes(0, 2)
    mask[:, :, :2] = (mask[:, :, :2] + 1) * 0.5
    P = PREDS_TTA if use_tta else PREDS
    pred = np.repeat(to_numpy(P[N]), 3, 0).swapaxes(0, 2)
    thr_mask = (pred > thr).astype(np.uint8)
    if adaptive_thr:
        thr_mask = cv2.threshold(
            (pred[:, :, 0] * 255).astype(np.uint8),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU
        )[1] / 255
#         thr_mask = cv2.adaptiveThreshold(
#             (pred[:, :, 0] * 255).astype(np.uint8), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 49, 7
#         ) / 255.
        thr_mask = np.stack([thr_mask,] * 3, axis=2)
#         thr_mask[pred < 1e-3] = 0
    score = jaccard_score(mask[:,:, 2], thr_mask[:,:,2], average="micro")
    if mask[:,:, 2].sum() == 0 and thr_mask[:,:,2].sum() == 0:
        score = 1
    stacked = np.hstack([img, mask, pred, thr_mask])
    plt.figure(figsize=(32,8))
    plt.imshow(stacked, cmap="gray")
    plt.title(f"Jaccard={score:.3f}")
    plt.axis("off")

interactive(children=(Dropdown(description='weights', options=('10.deeplab_2dtst_reduced_focal_20200303_171959…

# Test Results Exploration